In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import xlsxwriter as xw
import warnings
warnings.filterwarnings('ignore')
import openpyxl as op
from openpyxl import Workbook, load_workbook
from openpyxl.styles import PatternFill, Border, Side, Alignment, Protection, Font, Border, Side

In [2]:
input_loc = "/home/dhrubakumardolai/Desktop/Python Test/Rahul/Script 0 Split the dynamic file/Target/"
output_loc = "/home/dhrubakumardolai/Desktop/Python Test/Rahul/Script 0 Split the dynamic file/Result/"

In [3]:
result_loc = "/home/dhrubakumardolai/Desktop/Python Test/Rahul/Script 0 Split the dynamic file/Result/02-Dec-2022/ADD_EDIT/"

In [4]:
result_loc_2 = "/home/dhrubakumardolai/Desktop/Python Test/Rahul/Script 0 Split the dynamic file/Result/02-Dec-2022/DELETE/"

In [5]:
result_loc_3 = "/home/dhrubakumardolai/Desktop/Python Test/Rahul/Script 0 Split the dynamic file/Result/02-Dec-2022/"

In [6]:
file_size = 24999
batch_no_1 = 1
batch_no_2 = 1

In [7]:
target = pd.read_csv(input_loc+"/02-Dec/target.csv")

In [8]:
target.head(1)

,MerchantId,Objectives,IngestionStatus,s no,Report Date,reingestion no,Reingestion_original_date
0,BCR2DN6TSP67LXSI,Null,DELETE,2853044,25-Nov-2022,NaN,NaN


In [9]:
target['New_objective'] = target['Objectives'].str.contains(pat='Inactive')

In [10]:
target['New_objective'].fillna(False, inplace=True)

In [11]:
target['New_objective'].replace(True, "Inactive - INR 40 Extra Pay on Txn", inplace=True)
target['New_objective'].replace(False, np.nan, inplace=True)

In [12]:
target['New_objective'].fillna(target['Objectives'], inplace=True)

In [13]:
target.drop(['Objectives'],axis=1, inplace=True)

In [14]:
target.rename(columns={'New_objective':'Objectives'}, inplace=True)

In [17]:
target[["MerchantId", 'Objectives', 'IngestionStatus', 'reingestion no',
's no', 'Reingestion_original_date']].to_csv(output_loc+"Ingestion_objective_change.csv", index=False)

In [18]:
target_2 = target[["MerchantId", 'Objectives', 'IngestionStatus', 
        's no']]

In [19]:
target = target_2

In [20]:
target.shape

(281514, 4)

In [21]:
target['IngestionStatus'].value_counts()

ADD       144862
DELETE     84442
EDIT       52210
Name: IngestionStatus, dtype: int64

In [22]:
target['IngestionStatus'].unique()

array(['DELETE', 'ADD', 'EDIT'], dtype=object)

In [23]:
target_f_d = target[target['IngestionStatus'].isin(['DELETE'])]

In [24]:
target_f_d.to_csv(output_loc+"Split/Delete.csv", index=False)

In [25]:
target_f_A_E = target[target['IngestionStatus'].isin(['ADD', 'EDIT'])]

In [26]:
target_f_A_E.to_csv(output_loc+"Split/Add_Edit.csv", index=False)

In [27]:
for i in pd.read_csv(output_loc+"/Split/Add_Edit.csv", chunksize=file_size):
    i = i[~i['IngestionStatus'].isin(['DELETE'])]
    i.to_csv(result_loc+'ADD_EDIT'+str(batch_no_1)+'.csv', index=False)
    batch_no_1 += 1

In [28]:
for j in pd.read_csv(output_loc+"/Split/Delete.csv", chunksize=file_size):
    j = j[j['IngestionStatus'].isin(['DELETE'])]
    j.to_csv(result_loc_2+'Delete'+str(batch_no_2)+'.csv', index=False)
    batch_no_2 += 1

In [29]:
batch_no_3 = 1
batch_no_4 = 1
batch_no_5 = 1

In [30]:
# Delete Format
for i in pd.read_csv(output_loc+"/Split/Delete.csv", chunksize=file_size):
    i = i[i['IngestionStatus'].isin(['DELETE'])]
    i['Merchant ID'] = np.nan
    i.rename(columns={"MerchantId":'Merchant Ext.ID'}, inplace=True)
    i[['Merchant ID','Merchant Ext.ID']].to_excel(result_loc_3+'Delete_format/Delete_format'+str(batch_no_3)+'.xlsx', index=False)
    batch_no_3 += 1

In [31]:
# Adhoc Format
for i in pd.read_csv(output_loc+"/Split/Add_Edit.csv", chunksize=file_size):
    i = i[~i['IngestionStatus'].isin(['DELETE'])]
    i["Alternate Mobile"] = np.nan
    i.rename(columns={"MerchantId":'Merchant Ext.ID', "Objectives":'Eligibility'}, inplace=True)
    i[["Merchant Ext.ID", "Alternate Mobile", 
       "Eligibility"]].to_excel(result_loc_3+'AdHOC_format/AdHOC_format'+str(batch_no_4)+'.xlsx', index=False)
    batch_no_4 += 1

In [32]:
# Optional Format
for i in pd.read_csv(output_loc+"/Split/Add_Edit.csv", chunksize=file_size):
    i = i[~i['IngestionStatus'].isin(['DELETE'])]
    i.rename(columns={"MerchantId":'Merchant Ext.ID'}, inplace=True)
    i["Name"] = np.nan
    i["Last Transaction date"] = np.nan
    i["Phone Number"] = np.nan
    i["Address"] = np.nan
    i["Pincode"] = np.nan
    i["Latitude"] = np.nan
    i["Longitude"] = np.nan
    i["District"] = np.nan
    i["Polygon Name"] = np.nan
    i["Created By ID"] = np.nan
    i["Gpay OB E-mail"] = np.nan
    i["Merchant Health"] = np.nan
    i[["Merchant Ext.ID", 'Name', 'Phone Number', 'Address', 'Pincode', 'Latitude',
      'Longitude', 'District', 'Polygon Name', 'Created By ID', 'Last Transaction date',
      'Gpay OB E-mail', 'Merchant Health']].to_excel(result_loc_3+'Optional_format/Optional_format'+str(batch_no_5)+'.xlsx', index=False)
    batch_no_5 += 1